# The Oh My Information - Lab

In this assignment we take on, or really return to _taxic exploration_. We revisit the sniff world (aka _ScentGrid_), with a new twist. What happens when sense information is not just noisy, but suddenly missing altogether. A concrete case of this is turbulent flows. 

We'll be working with a crude approximation of such flows. Our environment this tims just deletes scent information from the grid, with a probability $(1- p_{scent})$. The noisy background is of course unaffected by this deletion.

The presence of this uncertainty makes decisions--of the kind common to decision theory--a necessity. So, we'll be using accumulator models, again. The decisions to be made are: 

- Q1: Is their a scent at all?
- Q2: Is the gradient increasing or decreasing? 

Our target metrics are number of deaths, total reward, and best reward. Any experimental trial which does not lead to finding at least a single target (aka reward) means the exploring agent dies. It's a harsh noisy world we live in, after all. 

## E Coli, again?
_Background_: Recall our basic model of E. Coli exploration is as simple as can be. 

- When the gradient is positive, meaning you are going "up" the gradient, the probability of turning is set to _p pos_. 
- When the gradient is negative, the turning probability is set to _p neg_. (See code below, for an example). 
- If the agent "decides" to turn, the direction it takes is uniform random.
- The length of travel before the next turn decision is sampled from an exponential distribution just like the _DiffusionDiscrete_

## Our agents, this time
We will study three agents. One who does _chemotaxis_. One who does a kind of _infotaxis_. One that does random search (aka Diffusion). For fun, let's call this one a _randotaxis_ agent. (No one really calls them this.) This last rando-agent is really a control. A reference point.

In a sense the _chemotaxis_ agent only tries to answer question Q2 (above). While _infotaxis_ only tries to answer Q1. They are extreme strategies, in other words. The bigger question we will ask, in a very limited setting, is which extreme methods is better _generally_? 

We will not address the question of combining them, into a less extreme and perhaps more reliable alternative...

## Costly cognition
Both _chemo-_ and _infotaxis_ agents will use a DDM-style accumulator to try and make better decisions about the direction of the gradient. These decisions are of course statistical in nature. (We won't be tuning the accumulator parameters in this lab. Assume the parameters I give you, for the DDM, are "good enough".)

For the _randotaxis_ agent number of steps means the number of steps or actions the agent takes. 

As in the _Air Quotes Lab_ we will assume that the steps are in a sense conserved. For the other two (accumulator) agents a step can mean two things. For accumulator agents a step can be spent sampling/weighing noisy scent evidence in the same location, or it can be spent moving to a new location.

## What's scent look like (a definition of _chemotaxis_)?
Our _chemotaxis_ agent (_AccumulatorGradientGrid_) tries to directly estimate the gradient $\nabla$ in scent by comparing the level of scent at the last grid position it occupied to the current scent level ($o$). By last grid position here we mean the last grid position when it moved last. 

$$\nabla \approx o_t - o_{t-1}$$

Because an accumulator is present, our chemo- sequentially tries to estimate this gradient by sampling the new current location, until the threshold is met.

 Chemo-accumulators have what we can think of as two cognitive or behavioral steps:

1. Use an accumulator to (stabely) estimate the chemo gradient
2. Use the gradient to make turning decisions

## More information (a definition of _infotaxis_)?
Compared to chemo- definition the definition of infotaxis is a little more involved. It has what we can think of as five cognitive or behavioral steps:

1. Use an accumulator to (stabely) estimate if there is a scent or not. AKA hits and misses.
2. Build a probability model of hits/misses (at every point)
3. Measure information gained when probability model changes 
4. Measure the gradient of information gains
5. Use the gradient to make turning decisions

_Note_: Even though the info-accumulator is more complex, it can take advantage of missing scent information to drive its behavior. It can also use positive scent hits, of course, too.

## Our TED talk moment
The big question for the week is this - is it ever better to follow the scent gradient instead of following its information?

## Install and import needed modules

In [ ]:
# Install explorationlib?
!pip install --upgrade git+https://github.com/parenthetical-e/explorationlib
!pip install --upgrade git+https://github.com/MattChanTK/gym-maze.git

In [ ]:
# Import misc
import shutil
import glob
import os
import copy
import sys

# Vis - 1
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Exp
from explorationlib.run import experiment
from explorationlib.util import select_exp
from explorationlib.util import load
from explorationlib.util import save

# Agents
from explorationlib.agent import GradientDiffusionGrid
from explorationlib.agent import AccumulatorGradientGrid

# Env
from explorationlib.local_gym import ScentGrid
from explorationlib.local_gym import create_grid_scent
from explorationlib.local_gym import uniform_targets
from explorationlib.local_gym import constant_values

# Vis - 2
from explorationlib.plot import plot_position2d
from explorationlib.plot import plot_length_hist
from explorationlib.plot import plot_length
from explorationlib.plot import plot_targets2d
from explorationlib.plot import plot_scent_grid

# Score
from explorationlib.score import total_reward
from explorationlib.score import num_death

In [ ]:
# Pretty plots
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.greedy=True
plt.rcParams["axes.facecolor"] = "white"
plt.rcParams["figure.facecolor"] = "white"
plt.rcParams["font.size"] = "16"

# Dev
%load_ext autoreload
%autoreload 2

## Section 1 -  Intelligent costs, for fun
### Random search versus Cognition, when information is often missing and noisy